Bson model Evaluation on the FRH04 region

In [ ]:
using BSON: @load
using NPZ, CSV, DataFrames
using Printf, Statistics
using Flux: Chain, Conv, Dense, BatchNorm, relu, Dropout, DataLoader, onecold, params

model_path = raw"/tempcnn_julia_model.bson"
x_path     = raw"/X_frh04.npy"
y_path     = raw"/y_true_frh04.npy"
fid_path   = raw"/field_ids_frh04.npy"
map_path   = raw"/classmapping_final_cleaned.csv"
max_rows   = 10 

println("Loading BSON model...")
@load model_path model_cpu
model = model_cpu

# === Input Data ===
println("Loading input data...")
X_raw = npzread(x_path)           
y_true = npzread(y_path)         
field_ids = npzread(fid_path)     
classmap = CSV.read(map_path, DataFrame)

# === Class name mapping ===
grouped = combine(groupby(classmap, :id), :classname => first => :classname)
classnames = sort(grouped, :id).classname

# === input ===
X = permutedims(Float32.(X_raw), (2, 3, 1))  
N = size(X, 3)
println("Data info:")
println("  Input shape: $(size(X_raw))")
println("  Labels range: $(minimum(y_true)) to $(maximum(y_true))")
println("  Number of classes: $(length(classnames))")

# === Run inference ===
println("Running inference on $N samples...")
logits = model(X)                
logits_t = permutedims(logits, (2, 1)) 

# === Apply softmax ===
exp_logits = exp.(logits_t .- maximum(logits_t, dims=2))
probs = exp_logits ./ sum(exp_logits, dims=2)
y_pred = [i[2] - 1 for i in argmax(probs, dims=2)] 

# === Compute accuracy ===
correct = sum(y_true .== y_pred)
accuracy = correct / length(y_true) * 100
@printf("\nOverall Accuracy: %.2f%%\n\n", accuracy)

# === Print sample results ===
println("Showing $max_rows samples...\n")
for i in 1:min(max_rows, N)
    fid = field_ids[i]
    y_t = y_true[i]
    y_p = y_pred[i]
    println("Field ID: $fid")
    println("   True Class Index: $y_t → $(classnames[y_t + 1])")
    println("   Predicted Class Index: $y_p → $(classnames[y_p + 1])")
    println("   Probabilities: $(round.(probs[i, :], digits=4))")
    println("------------------------------------------------------------")


end


Flux ONNX Model Evaluation on the FRH04 Region

In [ ]:
using ONNXRunTime
using NPZ, CSV, DataFrames
using Printf, Statistics

# === CONFIG ===
model_path = raw"/tempcnn_julia_model.onnx"
x_path     = raw"/X_frh04.npy"
y_path     = raw"/y_true_frh04.npy"
fid_path   = raw"/field_ids_frh04.npy"
map_path   = raw"/classmapping_final_cleaned.csv"
max_rows   = 10

# === Check files ===
for (label, path) in [("Model", model_path), ("Input", x_path), ("Labels", y_path),
                      ("IDs", fid_path), ("Mapping", map_path)]
    if !isfile(path)
        @error "$label file not found: $path"
        exit()
    end
end

# === Load model ===
println("Loading ONNX model...")
session = ONNXRunTime.load_inference(model_path)

# === Load data ===
X_raw = npzread(x_path)          
y_true = npzread(y_path)       
field_ids = npzread(fid_path)   
classmap = CSV.read(map_path, DataFrame)

# === Input  ===
X = Float32.(X_raw)
X = permutedims(X, (1, 3, 2))   
N = size(X, 1)

# === Class mapping  ===
grouped = combine(groupby(classmap, :id), :classname => first => :classname)
classnames = sort(grouped, :id).classname

# === Inference per sample ===
println("Running inference on $N samples ...")
y_pred = Int[]
probs_all = Array{Float32, 2}(undef, N, length(classnames))

for i in 1:N
    x_i = reshape(X[i, :, :], 1, size(X, 2), size(X, 3))  
    output = session(Dict("data_0" => x_i))
    logits = first(values(output))[1, :]  
    
    exp_logits = exp.(logits .- maximum(logits))
    probs = exp_logits ./ sum(exp_logits)
    
    push!(y_pred, argmax(probs) - 1)
    probs_all[i, :] = probs
end

# === Accuracy ===
correct = sum(y_true .== y_pred)
accuracy = correct / N * 100
@printf("\nOverall Accuracy: %.2f%%\n\n", accuracy)

# === Sample predictions ===
println("Showing $max_rows predictions:\n")
for i in 1:min(max_rows, N)
    fid = field_ids[i]
    y_t = y_true[i]
    y_p = y_pred[i]
    println("Field ID: $fid")
    println("   True Label: $y_t → $(classnames[y_t + 1])")
    println("   Predicted:  $y_p → $(classnames[y_p + 1])")
    println("   Probabilities: $(round.(probs_all[i, :], digits=4))")
    println("------------------------------------------------------------")
end

Compute RMSE and MAE

In [ ]:
using BSON: @load
using NPZ, CSV, DataFrames
using Flux
using ONNXRunTime
using Statistics, Printf

# need to specify the path
model_bson  = raw"/tempcnn_julia_model.bson"
x_path      = raw"/X_frh04.npy"            
y_path      = raw"/y_true_frh04.npy"       
fid_path    = raw"/field_ids_frh04.npy"    
map_path    = raw"/classmapping_final_cleaned.csv"
onnx_path   = raw"/tempcnn_julia_model.onnx"

out_native  = "native_logits_julia.npy"  
out_onnx    = "onnx_logits_julia.npy"   

# === LOAD DATA ===
println("Loading input data…")
X_raw   = npzread(x_path)   
y_true  = vec(npzread(y_path))::Vector{Int}      
fieldid = vec(npzread(fid_path))                 
N, T, Cin = size(X_raw)                     

classmap  = CSV.read(map_path, DataFrame)
classnames = sort(combine(groupby(classmap, :id), :classname => first => :classname), :id).classname
@printf "Data: N=%d, T=%d, Cin=%d, classes=%d\n" N T Cin length(classnames)


#  === LOAD JULIA MODEL (Flux) === 

println("Loading Flux model…")
@load model_bson model_cpu
model = model_cpu
Flux.testmode!(model)   

#  === LOAD ONNX SESSION  ===
println("Loading ONNX model…")
ENV["ONNXRuntime_DISABLE_CPU_AFFINITY"] = "1"
session = ONNXRunTime.load_inference(onnx_path)
inp_name = first(ONNXRunTime.input_names(session))


#  === INFERENCE ===

println("Running inference on $N samples…")

native_logits = Array{Float32}(undef, N, length(classnames))
onnx_logits   = Array{Float32}(undef, N, length(classnames))

for i in 1:N
    x_flux = permutedims(Float32.(X_raw[i, :, :]), (1,2)) 
    x_flux = reshape(x_flux, T, Cin, 1)                   
    logit_native = model(x_flux)                         
    native_logits[i, :] = vec(Array(logit_native))       

    x_onnx = permutedims(Float32.(X_raw[i, :, :]), (2,1)) 
    x_onnx = reshape(x_onnx, 1, Cin, T)                  
    out = session(Dict(inp_name => x_onnx))
    logit_onnx = first(values(out))                       
    onnx_logits[i, :] = vec(logit_onnx)                  
end


#  === SAVE LOGITS ===

npzwrite(out_native, native_logits)
npzwrite(out_onnx, onnx_logits)
println("Saved: ", out_native, " and ", out_onnx)

#  === QUICK FIDELITY CHECK  ===

mae    = mean(abs.(native_logits .- onnx_logits))
rmse   = sqrt(mean((native_logits .- onnx_logits).^2)) 
maxerr = maximum(abs.(native_logits .- onnx_logits))
top1   = mean(argmax.(eachrow(native_logits)) .== argmax.(eachrow(onnx_logits))) * 100
using LinearAlgebra
function cosine(a::AbstractVector, b::AbstractVector)
    na = norm(a); nb = norm(b)
    return (na == 0 || nb == 0) ? 1.0 : (dot(a,b) / (na*nb))
end
med_cos = median([cosine(native_logits[i, :], onnx_logits[i, :]) for i in 1:size(native_logits,1)])
@printf "Fidelity — MAE=%.2e, Max|err|=%.2e, Top1=%.4f%%\n" mae maxerr top1
@printf "Fidelity — MAE=%.8f, RMSE=%.8f, Max|err|=%.8f, Top1=%.4f%%, MedianCos=%.6f\n" mae rmse maxerr top1 med_cos

Prediction Results for the FRH04

In [ ]:
using BSON: @load
using NPZ, CSV, DataFrames
using Flux
using ONNXRunTime
using Statistics, Printf
using StatsBase
using Plots

model_bson  = raw"/tempcnn_julia_model.bson"
x_path      = raw"/X_frh04.npy"            
y_path      = raw"/y_true_frh04.npy"       
fid_path    = raw"/field_ids_frh04.npy"    
map_path    = raw"/classmapping_final_cleaned.csv"
onnx_path   = raw"/tempcnn_julia_model.onnx"

# will be used for maps visualization
out_samples_csv = "julia_sample_results.csv"

# LOAD DATA
println("Loading input data…")
X_raw   = npzread(x_path)                         
y_true  = vec(npzread(y_path))::Vector{Int}     
fieldid = vec(npzread(fid_path))
N, T, Cin = size(X_raw)

classmap   = CSV.read(map_path, DataFrame)
classmap_uniq = sort(combine(groupby(classmap, :id), :classname => first => :classname), :id)
classnames = classmap_uniq.classname
C = length(classnames)
@printf "Data: N=%d, T=%d, Cin=%d, classes=%d\n" N T Cin C
y_true_1 = minimum(y_true) == 0 ? (y_true .+ 1) : y_true

# LOAD JULIA MODEL (Flux)
println("Loading Flux model…")
@load model_bson model_cpu
model = model_cpu
Flux.testmode!(model)

# LOAD ONNX SESSION
println("Loading ONNX model…")
ENV["ONNXRuntime_DISABLE_CPU_AFFINITY"] = "1"
session = ONNXRunTime.load_inference(onnx_path)
inp_name = first(ONNXRunTime.input_names(session))

# INFERENCE 
println("Running inference on $N samples…")
pred_native = Vector{Int}(undef, N)       
pred_onnx   = Vector{Int}(undef, N)       
conf_native = Vector{Float32}(undef, N)   
conf_onnx   = Vector{Float32}(undef, N)
max_prob_diff = Vector{Float32}(undef, N) 
softmax_vec(v::AbstractVector) = begin
    m = maximum(v)
    ex = exp.(v .- m)
    ex ./ sum(ex)
end

for i in 1:N
    x_flux = permutedims(Float32.(X_raw[i, :, :]), (1,2))    
    x_flux = reshape(x_flux, T, Cin, 1)                      
    logit_native = vec(Array(model(x_flux)))                 
    p_nat = softmax_vec(logit_native)


    x_onnx = permutedims(Float32.(X_raw[i, :, :]), (2,1))    
    x_onnx = reshape(x_onnx, 1, Cin, T)                      
    out = session(Dict(inp_name => x_onnx))
    logit_onnx = vec(first(values(out)))                     
    p_onnx = softmax_vec(logit_onnx)

    # Predictions + confidences
    _max_nat, _arg_nat = findmax(p_nat)   
    _max_onnx, _arg_onnx = findmax(p_onnx)

    pred_native[i] = _arg_nat
    pred_onnx[i]   = _arg_onnx
    conf_native[i] = Float32(_max_nat)
    conf_onnx[i]   = Float32(_max_onnx)

    # max per-class prob difference
    max_prob_diff[i] = maximum(abs.(p_nat .- p_onnx)) |> Float32
end

#  PER-SAMPLE CSV
df_samples = DataFrame(
    sample_idx        = collect(1:N),
    field_id          = fieldid,
    ground_truth_id1  = y_true_1,
    ground_truth_name = classnames[y_true_1],
    julia_pred_id1    = pred_native,
    julia_pred_name   = classnames[pred_native],
    onnx_pred_id1     = pred_onnx,
    onnx_pred_name    = classnames[pred_onnx],
    predictions_match = pred_native .== pred_onnx,
    julia_correct     = pred_native .== y_true_1,
    onnx_correct      = pred_onnx .== y_true_1,
    julia_conf        = conf_native,
    onnx_conf         = conf_onnx,
    max_prob_diff     = max_prob_diff
)

CSV.write(out_samples_csv, df_samples)
println("Wrote per-sample results → $(out_samples_csv)")


PyTorch ONNX Evaluation on the FRH04 Region 

In [ ]:
using ONNXRunTime
using NPZ
using CSV
using DataFrames
using Printf

function main()
    # === CONFIG ===
    model_path = raw"python onnx model path/tempcnn_py_model.onnx"
    x_path     = raw"/X_frh04.npy"
    y_path     = raw"/y_true_frh04.npy"
    fid_path   = raw"/field_ids_frh04.npy"
    map_path   = raw"/classmapping_final_cleaned.csv"
    max_rows   = 10  
    for (label, path) in [("Model", model_path), ("Input", x_path), ("Labels", y_path),
                          ("IDs", fid_path), ("Mapping", map_path)]
        if !isfile(path)
            @error "$label file not found: $path"
            return
        end
    end

    println("Loading ONNX model...")
    model = ONNXRunTime.load_inference(model_path)

    println("Loading input data...")
    X = npzread(x_path)              
    y_true = npzread(y_path)         
    field_ids = npzread(fid_path)   
    classmap = CSV.read(map_path, DataFrame)

    # Extract and sort class names based on class index
    grouped = combine(groupby(classmap, :id), :classname => first => :classname)
    classnames = sort(grouped, :id).classname

    # === Run inference ===
    println("Running inference on $(size(X, 1)) samples...")
    input = Dict("input" => X)
    output_dict = model(input)
    logits = first(values(output_dict))

    # === Softmax + argmax ===
    exp_logits = exp.(logits .- maximum(logits, dims=2))
    probs = exp_logits ./ sum(exp_logits, dims=2)
    y_pred = [i[2] - 1 for i in argmax(probs, dims=2)]

    println("\nInference complete! Showing top $max_rows results:\n")
    correct = 0
    for i in 1:min(max_rows, length(y_pred))
        fid = field_ids[i]
        y_true_i = y_true[i]
        y_pred_i = y_pred[i]
        println("Field ID: $fid")
        println("   True Class Index: $y_true_i → $(classnames[y_true_i + 1])")
        println("   Predicted Class Index: $y_pred_i → $(classnames[y_pred_i + 1])")
        println("   Probabilities: $(round.(probs[i, :], digits=4))")
        println("------------------------------------------------------------")
        correct += (y_true_i == y_pred_i)
    end

    correct = sum(y_true .== y_pred)
    accuracy = correct / length(y_true) * 100
    @printf("Overall Accuracy: %.2f%%\n", accuracy)
end

main()